# Train Driving Model

In [1]:
import os
import csv

import cv2
import numpy as np
import pandas as pd
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
samples = []
with open('../data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    # Skip header line
    header = next(reader)
    for line in reader:
        samples.append(line)

In [3]:
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

In [4]:
def generator(samples, batch_size=32, correction=0):
    num_samples = len(samples) - (len(samples) % batch_size)
    shuffle(samples)
    while 1: # Loop forever so the generator never terminates
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name_center = '../data/IMG/'+batch_sample[0].split('/')[-1]
                name_left = '../data/IMG/'+batch_sample[1].split('/')[-1]
                name_right = '../data/IMG/'+batch_sample[2].split('/')[-1]
                
                image_center = cv2.imread(name_center)
                steering_center = float(batch_sample[3])
                image_left = cv2.imread(name_left)
                steering_left = steering_center + correction
                image_right = cv2.imread(name_right)
                steering_right = steering_center - correction
                
                #images.extend([image_center, image_left, image_right])
                #angles.extend([steering_center, steering_left, steering_right])
                images.extend([image_center])
                angles.extend([steering_center])
                
                # data augmentation
                #images.extend([np.fliplr(image_center), np.fliplr(image_left), np.fliplr(image_right)])
                #angles.extend([-steering_center, -steering_left, -steering_right])
                images.extend([np.fliplr(image_center)])
                angles.extend([-steering_center])

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

In [5]:
# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32, correction=0)
validation_generator = generator(validation_samples, batch_size=32, correction=0)

In [6]:
from keras.models import Sequential
from keras.layers import Cropping2D, Dense, Flatten, Lambda, Conv2D, Convolution2D, Dropout
from keras.layers.pooling import MaxPooling2D

Using TensorFlow backend.


In [7]:
ch, row, col = 3, 80, 320  # Trimmed image format

model = Sequential()
model.add(Lambda(lambda x: x / 255 - 0.5, input_shape=(160, 320, 3)))
model.add(Convolution2D(6, 5, 5, activation='relu', border_mode= 'same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Convolution2D(6, 5, 5, activation='relu', border_mode= 'same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

In [8]:
model.compile(loss='mse', optimizer='adam')
history = model.fit_generator(
    train_generator,
    samples_per_epoch= len(train_samples),
    validation_data=validation_generator, 
    nb_val_samples=len(validation_samples),
    nb_epoch=2,
) 

Epoch 1/2
5440/6428 [========================>.....] - ETA: 58s - loss: 0.0783

KeyboardInterrupt: 

In [31]:
model.save('./model.ver-3.h5')

In [32]:
history.history

{'loss': [0.040938242352429301, 0.014303119498129823],
 'val_loss': [0.016521635966805313, 0.01452325818200524]}